In [37]:
import pandas as pd
import numpy as np
import re
import requests
import time

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [38]:
df = pd.read_csv(r"C:\INNOMATICS RESEARCH LAB\7 ML\Projects\Text Projects\Politics_with_sentiment.csv")
df.head()

video_id                                            comment sentiment
0  9GscBMFnoi8  Dirty Politics? Are Modi and Amit clean? Haha....  Negative
1  9GscBMFnoi8  Why bengal didn't has local leadership. Can Sa...  Positive
2  9GscBMFnoi8  I believe in both NDA-ruled Maharastra & BJP-r...  Positive
3  9GscBMFnoi8  Because the BJP doesn't play foul, parties lik...   Neutral
4  9GscBMFnoi8  Cheap politics by Rahul and his Congress. Righ...  Positive

In [39]:
df=df.drop(columns=["video_id"])

In [40]:
df.duplicated().sum()

1374

In [41]:
df=df.drop_duplicates()

In [42]:
df.isnull().sum()

comment      0
sentiment    0
dtype: int64

In [43]:
import numpy as np
import re

def clean_comment(text):
    """Clean text data: handle arrays, lists, NaN, URLs, punctuation, digits."""
    
    if isinstance(text, (list, np.ndarray)):
        text = " ".join(map(str, text.flatten() if isinstance(text, np.ndarray) else text))
    text = str(text)
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df['comment'] = df['comment'].apply(
    lambda x: " ".join(map(str, x.flatten())) if isinstance(x, np.ndarray) 
    else " ".join(map(str, x)) if isinstance(x, list) 
    else str(x)
)
df['comment'] = df['comment'].apply(clean_comment)

In [44]:
y=df["sentiment"]

In [45]:
X=df['comment']

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5300,)
(5300,)
(1326,)
(1326,)


In [48]:
# Build text classification pipeline with TF-IDF and SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('svc', SVC(kernel='linear', C=1.0))
])

# Train only if labels exist
if y is not None:
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.83      0.34      0.48       197
     Neutral       0.70      0.90      0.78       648
    Positive       0.84      0.73      0.78       481

    accuracy                           0.75      1326
   macro avg       0.79      0.65      0.68      1326
weighted avg       0.77      0.75      0.74      1326



In [49]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.751131221719457


In [50]:
import pickle

with open("ML_Text.pkl","wb") as f:
    pickle.dump(pipeline,f)

In [51]:
with open("ML_Text.pkl","rb") as f:
    model = pickle.load(f)

In [52]:
model

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('svc', SVC(kernel='linear'))])

In [53]:
model.predict(["I think the Government is bad"])

array(['Negative'], dtype=object)

In [54]:
model.predict(["I think the Government is Not bad"])

array(['Negative'], dtype=object)